# Notes and excercises from *Structure and Interpretation of Computer Programs*

The book is [available online on MIT's site](https://mitpress.mit.edu/sites/default/files/sicp/full-text/book/book.html), there are also [recorded lectures on MIT's OCW](https://www.youtube.com/playlist?list=PLE18841CABEA24090).

I'm using the [Calysto Scheme](https://github.com/Calysto/calysto_scheme) notebooks.

```python
pip install calysto_scheme
```

## Chapter 1. Building Abstractions with Procedures

### `lambda` vs `define`

Definitions have the form

```scheme
(define <var> <value>)
```

or the form

```scheme
(define (<var> <parameter1> ... <parametern>)
  <body>)
```

The latter form (standard procedure definition) is syntactic sugar for

```scheme
(define <var>
  (lambda (<parameter1> ... <parametern>)
    <body>))
```

### `let`

The general form of a `let` expression is

```scheme
(let ((<var1> <exp1>)
      (<var2> <exp2>)
      
      (<varn> <expn>))
   <body>)
```

[...] the `let` expression is interpreted as an alternate syntax for

```scheme
((lambda (<var1> ...<varn>)
    <body>)
 <exp1>
 
 <expn>)
```

In [1]:
;; math functions missig from Calysto Scheme

(import "math")

(define sin math.sin)
(define cos math.cos)
(define tan math.tan)
(define atan math.atan)
(define log math.log)

In [2]:
(define (square x) (* x x))

(assert = (square 2) 4)
(assert = (square 3) 9)

ok

In [3]:
(define ε 0.01)

(define (approx x y)
  (< (abs (- x y)) ε))

(assert = (approx 1 1) #t)
(assert = (approx (+ 1 (/ ε 2)) 1) #t)
(assert = (approx (- 1 (/ ε 2)) 1) #t)
(assert = (approx 2 1) #f)
(assert = (approx (+ 1 (* ε 1.1)) 1) #f)

ok

In [4]:
(define π 3.1415)

(define (circle-area r)
  (* π (square r)))

(assert = (circle-area 1) π)
(assert approx (circle-area 2) 12.56)

ok

In [5]:
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x)
                 x)))

(define (improve guess x)
  (average guess (/ x guess)))

(define (average x y)
  (/ (+ x y) 2))

(define (good-enough? guess x)
  (< (abs (- (square guess) x)) ε))

(define (sqrt x)
  (sqrt-iter 1 x))

(assert approx (sqrt 4) 2)
(assert approx (sqrt 9) 3)

ok

In [6]:
(define (good-enough? guess x)
    (< (abs (- (square guess) x)) ε))

(define (improve guess x)
    (average guess (/ x guess)))

(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))

(assert approx (sqrt-iter 1 4) 2)
(assert approx (sqrt-iter 1 9) 3)

ok

We would like to localize the subprocedures, hiding them inside `sqrt` so that `sqrt` could coexist with other successive approximations, each having its own private good-enough? procedure. [...]  Such nesting of definitions, called *block structure*, is basically the right solution to the simplest name-packaging problem.  [...] Since `x` is bound in the definition of sqrt, the procedures `good-enough?`, `improve`, and `sqrt-iter`, which are defined internally to sqrt, are in the scope of `x`. Thus, it is not necessary to pass x explicitly to each of these procedures. Instead, we allow `x` to be a free variable in the internal definitions, as shown below. Then `x` gets its value from the argument with which the enclosing procedure sqrt is called. This discipline is called *lexical scoping*.

In [7]:
(define (sqrt x)
  (define (good-enough? guess)
    (< (abs (- (square guess) x)) ε))
  (define (improve guess)
    (average guess (/ x guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess)
        guess
        (sqrt-iter (improve guess))))
  (sqrt-iter 1.0))

(assert approx (sqrt 4) 2)
(assert approx (sqrt 9) 3)

ok

In [8]:
;; local scope

(define (foo)
  (define π 0)
  π)

(print (foo))
(print π)

0
3.1415


In [9]:
;; we can replace the methods easily

(define (bar x) (* 2 x))

(define (foo x) (bar x))

(foo 5)

10

In [10]:
(define (bar x) (* 100 x))

(foo 5)

500

**Exercise 1.6.**  Alyssa P. Hacker doesn't see why if needs to be provided as a special form. "Why can't I just define it as an ordinary procedure in terms of `cond`?" she asks. Alyssa's friend Eva Lu Ator claims this can indeed be done, and she defines a new version of `if`:

In [11]:
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

(assert = (new-if (= 2 3) 0 5) 5)
(assert = (new-if (= 1 1) 0 5) 0)

ok

In [12]:
(define (sqrt-iter-bad guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter-bad (improve guess x) x)))

;; (sqrt-iter-2 1 2)

**Exercise 1.7.**  The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how guess changes from one iteration to the next and to stop when the change is a very small fraction of the `guess`. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers? 

In [13]:
(define (good-enough? guess x)
    (< (/ (abs (- (square guess) x)) guess) ε))

(assert approx (sqrt-iter 1.0 4) 2)
(assert approx (sqrt-iter 1.0 9) 3)

ok

**Exercise 1.8.**  Newton's method for cube roots is based on the fact that if $y$ is an approximation to the cube root of $x$, then a better approximation is given by the value 

$$
\frac{x / y^2 + 2y}{3}
$$

Use this formula to implement a cube-root procedure analogous to the square-root procedure. (In section 1.3.4 we will see how to implement Newton's method in general as an abstraction of these square-root and cube-root procedures.) 

In [14]:
(define (cube x) (* x x x))

(define (good-enough? guess x)
    (< (/ (abs (- (cube guess) x)) guess) ε))

(define (improve guess x)
    (/ (+ (/ x (square guess)) (* 2 guess)) 3))

(define (cube-root x)
  (define (iter guess x)
    (if (good-enough? guess x)
        guess
        (iter (improve guess x) x)))
  (iter 1.0 x))

(assert approx (cube-root 27) 3)
(assert approx (cube-root (cube 18)) 18)

ok

In [15]:
%%time

;; Naive Fibonacci sequence implementation with exponential O(ϕ^n) time and O(n) space usage.
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ (fib (- n 1))
                 (fib (- n 2))))))

(fib 15)

Time: 1.018855333328247 seconds.



610

In [16]:
%%time

(define (fib-iter a b count)
  (if (= count 0)
      b
      (fib-iter (+ a b) a (- count 1))))

;; Iterative Fibonacci sequence implementation with linear O(n) time and O(1) space usage.
(define (fib n)
  (fib-iter 1 0 n))

(fib 15)

Time: 0.018756389617919922 seconds.



610

**Exercise 1.11.**  A function $f$ is defined by the rule that $f(n) = n$ if $n<3$ and $f(n) = f(n - 1) + 2f(n - 2) + 3f(n - 3)$ if $n> 3$. Write a procedure that computes $f$ by means of a recursive process. Write a procedure that computes $f$ by means of an iterative process. 

In [17]:
(define (wsum fn1 fn2 fn3)
  (+ fn1 (* 2 fn2) (* 3 fn3)))

(define (f n)
  (if (< n 3) n
      (wsum (f (- n 1)) (f (- n 2)) (f (- n 3)))))

(assert = (f 1) 1)
(assert = (f 2) 2)
(assert = (f 3) 4)
(assert = (f 4) 11)
(assert = (f 5) 25)

ok

In [18]:
%%time
(f 15)

Time: 2.8630692958831787 seconds.



142717

In [19]:
(define (f n)
  (define (recur i fn1 fn2 fn3)
    (cond
     ((> i n) fn1)
     ((< i 3) (recur (+ i 1) i fn1 fn2))
     (else    (recur (+ i 1) (wsum fn1 fn2 fn3) fn1 fn2))))
  (recur 1 0 0 0))

(assert = (f 1) 1)
(assert = (f 2) 2)
(assert = (f 3) 4)
(assert = (f 4) 11)
(assert = (f 5) 25)

ok

In [20]:
%%time
(f 15)

Time: 0.012685298919677734 seconds.



142717

**Exercise 1.12.**  The following pattern of numbers is called Pascal's triangle.

```
    1
   1 1
  1 2 1
 1 3 3 1
1 4 6 4 1
```

The numbers at the edge of the triangle are all 1, and each number inside the triangle is the sum of the two numbers above it. Write a procedure that computes elements of Pascal's triangle by means of a recursive process. 

In [21]:
;; 0 1 0
;; 0 1 1 0
;; 0 1 2 1 0
;; 0 1 3 3 1 0
;; 0 1 2 6 4 1 0

(define (pascal-tri r c)
  (cond
   ((or (< c 1) (> c r)) 0)
   ((= r 1) 1)
   (else (+
          (pascal-tri (- r 1) (- c 1))
          (pascal-tri (- r 1) c)))))

(assert = (pascal-tri 1 1) 1)
(assert = (pascal-tri 2 1) 1)
(assert = (pascal-tri 2 2) 1)
(assert = (pascal-tri 2 3) 0)
(assert = (pascal-tri 5 3) 6)
(assert = (pascal-tri 5 4) 4)

ok

In [22]:
;; 1.2.4  Exponentiation

(define (expt b n)
  (expt-iter b n 1))

(define (expt-iter b counter product)
  (if (= counter 0)
      product
      (expt-iter b
                (- counter 1)
                (* b product)))) 

This method works fine for exponents that are powers of 2. We can also take advantage of successive squaring in computing exponentials in general if we use the rule

$$
b^n = \begin{cases}
(b^{n/2})^2        & \text{when } n \text{ is even} \\
b \cdot (b^{n -1}) & \text{when } n \text{ is odd} 
\end{cases}
$$

 We can express this method as a procedure:

In [23]:
(define (fast-expt b n)
  (cond ((= n 0) 1)
        ((even? n) (square (fast-expt b (/ n 2))))
        (else (* b (fast-expt b (- n 1))))))

(define (even? n)
  (= (remainder n 2) 0))

In [24]:
%%time
(assert = (fast-expt 45 541) (expt 45 541))

Time: 0.2945683002471924 seconds.



ok

**Exercise 1.16.**  Design a procedure that evolves an iterative exponentiation process that uses successive squaring and uses a logarithmic number of steps, as does fast-expt. (Hint: Using the observation that $(b^{n/2})^2 = (b^2)^{n/2}$, keep, along with the exponent $n$ and the base $b$, an additional state variable $a$, and define the state transformation in such a way that the product $ab^n$ is unchanged from state to state. At the beginning of the process $a$ is taken to be $1$, and the answer is given by the value of $a$ at the end of the process. In general, the technique of defining an invariant quantity that remains unchanged from state to state is a powerful way to think about the design of iterative algorithms.) 

In [25]:
;; a * b^n = a * (b^(n/2))^2
;; b^(n+1) = b * (b^(n/2))^2

(define (iter-expt b n a)
  (cond
   ((= n 0) a)
   ((even? n) (iter-expt (square b) (/ n 2) a))
   (else      (iter-expt b (- n 1) (* a b)))))

(assert = (iter-expt 7 1 1) 7)
(assert = (iter-expt 2 2 1) 4)
(assert = (iter-expt 5 7 1) (int (expt 5 7)))
(assert = (iter-expt 5 10 1) (int (expt 5 10)))

ok

In [26]:
%%time
(assert = (iter-expt 45 541 1) (expt 45 541))

Time: 0.2617988586425781 seconds.



ok

**Exercise 1.17.**  The exponentiation algorithms in this section are based on performing exponentiation by means of repeated multiplication. In a similar way, one can perform integer multiplication by means of repeated addition. The following multiplication procedure (in which it is assumed that our language can only add, not multiply) is analogous to the `expt` procedure:

```scheme
(define (* a b)
  (if (= b 0)
      0
      (+ a (* a (- b 1)))))
```

This algorithm takes a number of steps that is linear in $b$. Now suppose we include, together with addition, operations `double`, which doubles an integer, and `halve`, which divides an (even) integer by 2. Using these, design a multiplication procedure analogous to `fast-expt` that uses a logarithmic number of steps. 

In [27]:
(define (add* a b)
  (if (= b 0)
      0
      (+ a (add* a (- b 1)))))

In [28]:
%%time
(assert = (add* 532 157) (add* 157 532))
(assert = (add* 1532 3157) (add* 3157 1532))

Time: 2.6817572116851807 seconds.



ok

In [29]:
(define (double x) (+ x x))
(define (halve x) (/ x 2))

(define (fast* a b)
  (cond
   ((= b 1) a)
   ((even? b) (fast* (double a) (halve b)))
   (else (+ a (fast* a (- b 1))))))

(assert = (fast* 1 2) (* 1 2))
(assert = (fast* 2 2) (* 2 2))
(assert = (fast* 5 17) (* 5 17))
(assert = (fast* 3 21) (* 3 21))

ok

In [30]:
%%time
(assert = (fast* 532 157) (fast* 157 532))
(assert = (fast* 1532 3157) (fast* 3157 1532))

Time: 0.052290916442871094 seconds.



ok

**Exercise 1.18.**  Using the results of exercises 1.16 and 1.17, devise a procedure that generates an iterative process for multiplying two integers in terms of adding, doubling, and halving and uses a logarithmic number of steps.

In [31]:
(define (fast* a b c)
  (cond
   ((= b 0) c)
   ((even? b) (fast* (double a) (halve b) c))
   (else      (fast* a (- b 1) (+ a c)))))

(assert = (fast* 1 2 0) (* 1 2))
(assert = (fast* 2 2 0) (* 2 2))
(assert = (fast* 5 17 0) (* 5 17))
(assert = (fast* 3 21 0) (* 3 21))

ok

**Exercise 1.19.**   There is a clever algorithm for computing the Fibonacci numbers in a logarithmic number of steps. Recall the transformation of the state variables $a$ and $b$ in the `fib-iter` process of section 1.2.2: $a \leftarrow a + b$ and $b \leftarrow a$. Call this transformation $T$, and observe that applying $T$ over and over again n times, starting with $1$ and $0$, produces the pair $Fib(n + 1)$ and $Fib(n)$. In other words, the Fibonacci numbers are produced by applying $T^n$, the $n$-th power of the transformation $T$, starting with the pair $(1,0)$. Now consider $T$ to be the special case of $p = 0$ and $q = 1$ in a family of transformations $T_{pq}$, where $T_{pq}$ transforms the pair $(a,b)$ according to $a \leftarrow bq + aq + ap$ and $b \leftarrow bp + aq$. Show that if we apply such a transformation $T_{pq}$ twice, the effect is the same as using a single transformation $T_{p'q'}$ of the same form, and compute $p'$ and $q'$ in terms of $p$ and $q$. This gives us an explicit way to square these transformations, and thus we can compute $T^n$ using successive squaring, as in the `fast-expt` procedure. 

In [32]:
(define (fib n)
  (fib-iter 1 0 0 1 n))

(define (fib-iter a b p q count)
  (cond ((= count 0) b)
        ((even? count)
         (fib-iter a
                   b
                   (+ (* p p) (* q q))
                   (+ (* 2 q p) (* q q))
                   (/ count 2)))
        (else (fib-iter (+ (* b q) (* a q) (* a p))
                        (+ (* b p) (* a q))
                        p
                        q
                        (- count 1)))))

(assert = (fib 0) 0)
(assert = (fib 1) 1)
(assert = (fib 2) 1)
(assert = (fib 3) 2)
(assert = (fib 4) 3)
(assert = (fib 5) 5)
(assert = (fib 6) 8)

ok

In [33]:
;; 1.2.5  Greatest Common Divisors

(define (gcd a b)
  (if (= b 0)
      a
      (gcd b (remainder a b))))

(define (smallest-divisor n)
  (find-divisor n 2))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))))

(define (divides? a b)
  (= (remainder b a) 0))

(define (prime? n)
  (= n (smallest-divisor n)))

**Exercise 1.21.**  Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: 199, 1999, 19999. 

In [34]:
%%time
(smallest-divisor 199)
(smallest-divisor 1999)
(smallest-divisor 19999)

Time: 0.06247401237487793 seconds.



7

**Exercise 1.23.**  The `smallest-divisor` procedure shown at the start of this section does lots of needless testing: After it checks to see if the number is divisible by 2 there is no point in checking to see if it is divisible by any larger even numbers. This suggests that the values used for `test-divisor` should not be 2, 3, 4, 5, 6, ..., but rather 2, 3, 5, 7, 9, .... To implement this change, define a procedure next that returns 3 if its input is equal to 2 and otherwise returns its input plus 2. Modify the `smallest-divisor` procedure to use `(next test-divisor)` instead of `(+ test-divisor 1)`. With `timed-prime-test` incorporating this modified version of `smallest-divisor`, run the test for each of the 12 primes found in exercise 1.22. Since this modification halves the number of test steps, you should expect it to run about twice as fast. Is this expectation confirmed? If not, what is the observed ratio of the speeds of the two algorithms, and how do you explain the fact that it is different from 2? 

In [35]:
(define (next test-divisor)
  (if (< test-divisor 3)
      (+ test-divisor 1)
      (+ test-divisor 2)))

(assert = (next 1) 2)
(assert = (next 2) 3)
(assert = (next 3) 5)
(assert = (next 5) 7)

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (next test-divisor)))))

(assert = (smallest-divisor 1) 1)
(assert = (smallest-divisor 2) 2)
(assert = (smallest-divisor 4) 2)
(assert = (smallest-divisor 19999) 7)

ok

In [36]:
%%time
(smallest-divisor 199)
(smallest-divisor 1999)
(smallest-divisor 19999)

Time: 0.03593564033508301 seconds.



7

In [37]:
;; 1.3.1  Procedures as Arguments

(define (sum term a next b)
  (if (> a b)
      0
      (+ (term a)
         (sum term (next a) next b))))

(define (integral f a b dx)
  (define (add-dx x) (+ x dx))
  (* (sum f (+ a (/ dx 2.0)) add-dx b)
     dx))

(print (integral cube 0 1 0.01))
(print (integral cube 0 1 0.001))

;;  (The exact value of the integral of cube between 0 and 1 is 1/4.)

0.24998750000000042
0.249999875000001


**Exercise 1.29.**  Simpson's Rule is a more accurate method of numerical integration than the method illustrated above. Using Simpson's Rule, the integral of a function $f$ between $a$ and $b$ is approximated as 

$$
\frac{h}{3} \big[ y_0 + 4y_1 + 2y_2 + 4y_3 + 2y_4 + \dots + 2y_{n-2} + 4y_{n-1} + y_n \big]
$$

where $h = (b - a)/n$, for some even integer $n$, and $y_k = f(a + kh)$. (Increasing $n$ increases the accuracy of the approximation.) Define a procedure that takes as arguments $f$, $a$, $b$, and $n$ and returns the value of the integral, computed using Simpson's Rule. Use your procedure to integrate cube between $0$ and $1$ (with $n = 100$ and $n = 1000$), and compare the results to those of the integral procedure shown above. 

In [38]:
(define (simpsons-rule f a b n)
  (define h (/ (- b a) n))
  (define (dx k) (+ a (* k h)))
  (define (iter i acc)
    (if (>= (dx i) b)
        acc
        (iter
         (+ i 1)
         (+ acc
            (* (if (even? i) 2 4)
               (f (dx i)))))))
  (* (/ h 3)
     (+ (f a) (iter 1 0.0) (f b))))

(print (simpsons-rule cube 0.0 1.0 100))
(print (simpsons-rule cube 0.0 1.0 1000))

0.25000000000000006
0.25000000000000006


In [39]:
;; https://codology.net/post/sicp-solution-exercise-1-29/
;; assuming n is even
(define (simpsons-rule f a b n)
  (define h (/ (- b a) n))
  (define (add-2h x) (+ x h h))
  (* (+ (f a)
        (* 2 (sum f a       add-2h b))
        (* 4 (sum f (+ a h) add-2h b))
        (f b))
     (/ h 3)))

(print (simpsons-rule cube 0.0 1.0 100))
(print (simpsons-rule cube 0.0 1.0 1000))

0.25000000000000044
0.25000000000000083


**Exercise 1.30.**  The `sum` procedure above generates a linear recursion. The procedure can be rewritten so that the `sum` is performed iteratively. 

In [40]:
(define (sum term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter
         (next a)
         (+ result (term a)))))
  (iter a 0.0))

(define (identity x) x)
(define (inc x) (+ x 1))

(assert = (sum identity 1 inc 10) 55)
(assert = (sum square 1 inc 10) 385)

ok

**Exercise 1.31.**  
 a.  The sum procedure is only the simplest of a vast number of similar abstractions that can be captured as higher-order procedures. Write an analogous procedure called `product` that returns the product of the values of a function at points over a given range. Show how to define factorial in terms of product. Also use product to compute approximations to using the formula

$$
\frac{\pi}{4} = \frac{2 \cdot 4 \cdot 4 \cdot 6 \cdot 6 \cdot 8 \cdot \dots}{3 \cdot 3 \cdot 5 \cdot 5 \cdot 7 \cdot 7 \cdot \dots}
$$

 b.  If your `product` procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process. 

In [41]:
(define (product term a next b)
  (define (iter a)
    (if (> a b)
        1.0
         (* (term a)
            (iter (next a)))))
  (iter a))

(assert = (product identity 1 inc 5) 120)
(assert = (product square 1 inc 5) 14400)

ok

In [42]:
(define (product term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter
         (next a)
         (* result (term a)))))
  (iter a 1.0))

(assert = (product identity 1 inc 5) 120)
(assert = (product square 1 inc 5) 14400)

ok

In [43]:
(define (factorial n)
  (product identity 1 inc n))

(assert = (factorial 5) 120)

ok

In [44]:
;; https://en.wikipedia.org/wiki/Wallis_product

(define (wallis-product n)
  (define (f n) 
    (define 2n (* 2 n))
    (* (/ 2n (- 2n 1))
       (/ 2n (+ 2n 1))))
  (/ (product f 1.0 inc n) 2))

(assert approx (wallis-product 100) (/ π 4))

ok

**Exercise 1.32.**  a. Show that `sum` and `product` (exercise 1.31) are both special cases of a still more general notion called `accumulate` that combines a collection of terms, using some general accumulation function:

```scheme
(accumulate combiner null-value term a next b)
```

`Accumulate` takes as arguments the same term and range specifications as `sum` and `product`, together with a combiner procedure (of two arguments) that specifies how the current term is to be combined with the accumulation of the preceding terms and a null-value that specifies what base value to use when the terms run out. Write accumulate and show how sum and product can both be defined as simple calls to accumulate.

b. If your `accumulate` procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.

In [45]:
(define (accumulate combiner null-value term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter
         (next a)
         (combiner result (term a)))))
  (iter a null-value))

(assert = (accumulate + 0 identity 1 inc 5) (sum identity 1 inc 5))
(assert = (accumulate * 1 identity 1 inc 5) (product identity 1 inc 5))
(assert = (accumulate + 0 square 1 inc 5) (sum square 1 inc 5))
(assert = (accumulate * 1 square 1 inc 5) (product square 1 inc 5))

ok

**Exercise 1.33.**  You can obtain an even more general version of accumulate (exercise 1.32) by introducing the notion of a filter on the terms to be combined. That is, combine only those terms derived from values in the range that satisfy a specified condition. The resulting `filtered-accumulate` abstraction takes the same arguments as `accumulate`, together with an additional predicate of one argument that specifies the filter. Write `filtered-accumulate` as a procedure. Show how to express the following using `filtered-accumulate`:

a. the sum of the squares of the prime numbers in the interval $a$ to $b$ (assuming that you have a `prime?` predicate already written)

b. the product of all the positive integers less than n that are relatively prime to n (i.e., all positive integers $i < n$ such that $\mathrm{GCD}(i,n) = 1$). 

In [46]:
(define (filtered-accumulate accept-rule combiner null-value term a next b)
  (define (iter a result)
    (cond
     ((> a b) result)
     ((accept-rule a)
      (iter
       (next a)
       (combiner result (term a))))
     (else
      (iter (next a) result))))
  (iter a null-value))

(assert = (filtered-accumulate even? + 0 identity 1 inc 10) 30)
(assert = (filtered-accumulate even? + 0 square 1 inc 10) 220)
(assert = (filtered-accumulate prime? + 0 square 2 inc 20) 1027)

ok

In [47]:
(define (product-of-relative-primes n)
  (define (relative-prime? i)
    (= (gcd i n) 1))
  (filtered-accumulate relative-prime? * 1 identity 1 inc n))

(product-of-relative-primes 10)

189

In [48]:
;; 1.3.3  Procedures as General Methods

(define (positive? x) (>= 0 x))
(define (negative? x) (not (positive? x)))

(define (search f neg-point pos-point)
  (let ((midpoint (average neg-point pos-point)))
    (if (close-enough? neg-point pos-point)
        midpoint
        (let ((test-value (f midpoint)))
          (cond ((positive? test-value)
                 (search f neg-point midpoint))
                ((negative? test-value)
                 (search f midpoint pos-point))
                (else midpoint))))))

(define (close-enough? x y)
  (< (abs (- x y)) 0.001))

(define (half-interval-method f a b)
  (let ((a-value (f a))
        (b-value (f b)))
    (cond ((and (negative? a-value) (positive? b-value))
           (search f a b))
          ((and (negative? b-value) (positive? a-value))
           (search f b a))
          (else
           (error "Values are not of opposite sign" a b)))))

(print (half-interval-method sin 2.0 4.0))
(print (half-interval-method (lambda (x) (- (* x x x) (* 2 x) 3)) 1.0 2.0))

3.14111328125
1.89306640625


In [49]:
;; needed to rename `try` to `iter` since Calypso Scheme did not allow for re-defining `try`

(define tolerance 0.00001)

(define (fixed-point f first-guess)
  (define (close-enough? x y)
    (< (abs (- x y)) tolerance))
  (define (iter guess)
    (let ((next (f guess)))
      (if (close-enough? guess next)
          next
          (iter next))))
  (iter first-guess))

(print (fixed-point cos 1.0))
(print (fixed-point (lambda (y) (+ (sin y) (cos y))) 1.0))

0.7390822985224024
1.2587315962971173


**Exercise 1.35.**  Show that the golden ratio $\phi$ (section 1.2.2) is a fixed point of the transformation $x \to 1 + 1/x$, and use this fact to compute by means of the `fixed-point` procedure. 

In [50]:
(fixed-point (lambda (x) (+ 1 (/ 1 x))) 1.0)

1.6180327868852458

**Exercise 1.37.**

$$
\frac{N_1}{D_1 + \frac{N_2}{D_2 + \frac{N_3}{D_3 + \dots}}} \approx
\frac{N_1}{D_1 + \frac{N_2}{\ddots + \frac{N_k}{D_k}}}
$$

[...] Suppose that $n$ and $d$ are procedures of one argument (the term index i) that return the $N_i$ and $D_i$ of the terms of the continued fraction. Define a procedure `cont-frac` such that evaluating `(cont-frac n d k)` computes the value of the $k$-term finite continued fraction. Check your procedure by approximating $1/\phi$ using

```scheme
(cont-frac (lambda (i) 1.0)
           (lambda (i) 1.0)
           k)
```

for successive values of $k$. How large must you make $k$ in order to get an approximation that is accurate to 4 decimal places?

b. If your cont-frac procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process. 

In [51]:
(define (cont-frac N D k)
  (define (iter i)
    (if (> i k)
        (/ (N i) (D i))
        (/ (N i)
           (+ (D i)
              (iter (+ i 1))))))
   (iter 1))

(define ϕ 1.6180)

(assert
 (lambda (x y) (< (abs (- x y)) .0001))
 (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 10)
 (/ 1 ϕ))

ok

In [52]:
(define (cont-frac N D k)
  (define (iter i acc)
    (if (= i 0)
        acc
        (iter
         (- i 1)
         (/ (N i)
            (+ (D i) acc)))))
  (iter
   (- k 1)
   (/ (N k) (D k))))

(assert
 (lambda (x y) (< (abs (- x y)) .0001))
 (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 10)
 (/ 1 ϕ))

ok

**Exercise 1.38.**  In 1737, the Swiss mathematician Leonhard Euler published a memoir *De Fractionibus Continuis*, which included a continued fraction expansion for $e - 2$, where $e$ is the base of the natural logarithms. In this fraction, the $N_i$ are all $1$, and the $D_i$ are successively $1, 2, 1, 1, 4, 1, 1, 6, 1, 1, 8, \dots$. Write a program that uses your `cont-frac` procedure from exercise 1.37 to approximate $e$, based on Euler's expansion. 

In [53]:
(define (Di i)
  (if (= 2 (modulo i 3))
      (* 2(/ (+ i 1) 3))
      1))

(assert = (Di 1) 1)
(assert = (Di 2) 2)
(assert = (Di 3) 1)
(assert = (Di 4) 1)
(assert = (Di 5) 4)
(assert = (Di 8) 6)

ok

In [54]:
(define (euler-exp n)
  (cont-frac (lambda (i) 1.0) Di n))

(define e 2.71828)

(assert approx (euler-exp 5) (- e 2))

ok

**Exercise 1.39.**  A continued fraction representation of the tangent function was published in 1770 by the German mathematician J.H. Lambert:

$$
\operatorname{tan} r = \frac{r}{1 - \frac{r^2}{2 - \frac{r^2}{3 - \dots}}}
$$

where $x$ is in radians. Define a procedure `(tan-cf x k)` that computes an approximation to the tangent function based on Lambert's formula. $K$ specifies the number of terms to compute, as in exercise 1.37. 

In [55]:
(define (tan-cf x k)
  (cont-frac (lambda (i) (if (= i 1) x (* x x -1)))
             (lambda (i) (- (* 2.0 i) 1))
             k))

(assert approx (tan-cf 1 10) 1.5574)
(assert approx (tan-cf 2 10) -2.1850)

ok

In [56]:
;; 1.3.4  Procedures as Returned Values - Newton's method

(define (deriv g)
  (lambda (x)
    (/ (- (g (+ x dx)) (g x))
       dx)))

(define dx 0.00001)

((deriv cube) 5)

75.00014999664018

In [57]:
(define (newton-transform g)
  (lambda (x)
    (- x (/ (g x) ((deriv g) x)))))

(define (newtons-method g guess)
  (fixed-point (newton-transform g) guess))

**Exercise 1.40.**  Define a procedure cubic that can be used together with the newtons-method procedure in expressions of the form

```scheme
(newtons-method (cubic a b c) 1)
```

to approximate zeros of the cubic $x^3 + ax^2 + bx + c$. 

In [58]:
(define (cubic a b c)
  (lambda (x) (+
               (expt x 3)
               (* a (expt x 2))
               (* b x)
               c)))

(newtons-method (cubic 0 0 1) 1)

-0.9999999999999863

**Exercise 1.41.**  Define a procedure `double` that takes a procedure of one argument as argument and returns a procedure that applies the original procedure twice. For example, if `inc` is a procedure that adds 1 to its argument, then `(double inc)` should be a procedure that adds 2. What value is returned by

```scheme
(((double (double double)) inc) 5)
```

In [59]:
(define (double f)
  (lambda (x) (f (f x))))

(assert = ((double inc) 1) 3)

(((double (double double)) inc) 5)

21

**Exercise 1.42.**  Let $f$ and $g$ be two one-argument functions. The composition $f$ after $g$ is defined to be the function $x \to f(g(x))$. Define a procedure compose that implements composition.

In [60]:
(define (compose f g)
  (lambda (x) (f (g x))))

(assert = ((compose square inc) 6) 49)

ok

**Exercise 1.43.**  If $f$ is a numerical function and $n$ is a positive integer, then we can form the $n$th repeated application of $f$, which is defined to be the function whose value at $x$ is $f(f(...(f(x))...))$. For example, if $f$ is the function $x \to x + 1$, then the nth repeated application of $f$ is the function $x \to x + n$. If $f$ is the operation of squaring a number, then the nth repeated application of $f$ is the function that raises its argument to the 2nth power. Write a procedure that takes as inputs a procedure that computes $f$ and a positive integer $n$ and returns the procedure that computes the nth repeated application of $f$. 

In [61]:
(define (repeated f n)
  (lambda (x)
    (define (iter i)
      (if (> i n)
          x
          (f (iter (+ i 1)))))
    (iter 1)))

(assert = ((repeated square 2) 5) 625)

ok

**Exercise 1.44.**  The idea of smoothing a function is an important concept in signal processing. If $f$ is a function and $dx$ is some small number, then the smoothed version of $f$ is the function whose value at a point $x$ is the average of $f(x - dx)$, $f(x)$, and $f(x + dx)$. Write a procedure `smooth` that takes as input a procedure that computes $f$ and returns a procedure that computes the smoothed $f$. It is sometimes valuable to repeatedly smooth a function (that is, smooth the smoothed function, and so on) to obtained the $n$-fold smoothed function. Show how to generate the $n$-fold smoothed function of any given function using `smooth` and `repeated` from exercise 1.43.

In [62]:
(define (smooth f)
  (lambda (x)
    (/ (+
        (f (- x ε))
        (f x)
        (f (+ x ε)))
       3)))

(define (smooth-n-fold f n)
  ((repeated smooth n) f))

**Exercise 1.45.**  We saw in section 1.3.3 that attempting to compute square roots by naively finding a fixed point of $y \to x/y$ does not converge, and that this can be fixed by average damping. The same method works for finding cube roots as fixed points of the average-damped $y \to x/y^2$. Unfortunately, the process does not work for fourth roots -- a single average damp is not enough to make a fixed-point search for $y \to x/y^3$ converge. On the other hand, if we average damp twice (i.e., use the average damp of the average damp of $y \to x/y^3$) the fixed-point search does converge. Do some experiments to determine how many average damps are required to compute nth roots as a fixed-point search based upon repeated average damping of $y \to x/y^{n-1}$. Use this to implement a simple procedure for computing $n$th roots using `fixed-point`, `average-damp`, and the `repeated` procedure of exercise 1.43. Assume that any arithmetic operations you need are available as primitives.

**Exercise 1.46.**  Several of the numerical methods described in this chapter are instances of an extremely general computational strategy known as iterative improvement. Iterative improvement says that, to compute something, we start with an initial guess for the answer, test if the guess is good enough, and otherwise improve the guess and continue the process using the improved guess as the new guess. Write a procedure `iterative-improve` that takes two procedures as arguments: a method for telling whether a guess is good enough and a method for improving a guess. `Iterative-improve` should return as its value a procedure that takes a guess as argument and keeps improving the guess until it is good enough. Rewrite the `sqrt` procedure of section 1.1.7 and the fixed-point procedure of section 1.3.3 in terms of `iterative-improve`. 